In [219]:
import pandas as pd
import psycopg2
import os
import plotly.express as px

In [ ]:
horse_df = pd.read_csv('data/horse_db.csv')
race_df = pd.read_csv('data/horse_race_data_db.csv')
race_df2 = pd.read_csv('data/08_01_to_11_30_horse_race_data.csv')
race_df3 = pd.read_csv('data/horse_race_data_db_before_08_01_21.csv')
race_meta_df = pd.read_csv('data/race_db.csv')
race_meta_df2 = pd.read_csv('data/08_01_to_11_30_race.csv')

In [188]:
conn = psycopg2.connect(host='singapore-postgres.render.com', database='zed_0t9u', user='zed_user', password='eNgTnw7LHNPuFa23C89h5SqeGqlRlRTY')

In [ ]:
def create_table(conn, query):
    cursor = conn.cursor()
    cursor.execute(query)
    conn.commit()
    cursor.close()

In [ ]:
horse_make_table = """CREATE TABLE IF NOT EXISTS horses (
                        horse_id integer PRIMARY KEY
                        , name varchar(250)
                        , genotype varchar(5)
                        , bloodline varchar(10)
                        , breed_type varchar(10)
                        , colour varchar(25)
                        , super_coat varchar(2)
                        , mother varchar(15)
                        , father varchar(15)
                        , horse_type varchar(10)
                        , birthday varchar(25)
                        , address varchar(50)
                        , stable_name varchar(50)
);"""
race_make_table = """CREATE TABLE IF NOT EXISTS races (
                        race_id varchar(10)
                        , horse_id float
                        , fire integer
                        , place integer
                        , horse_time float
                        , gate float
);"""
race_meta_make_table = """CREATE TABLE IF NOT EXISTS races_info (
                        race_id varchar(10)
                        , start_time varchar(25)
                        , name varchar(50)
                        , event_type varchar(50)
                        , distance float
                        , class int
                        , country varchar(25)
                        , city varchar(30)
                        , weather varchar(10)
                        , fee float
                        , prize_pool_first float
                        , prize_pool_second float
                        , prize_pool_third float
);"""

In [ ]:
create_table(conn, horse_make_table)
create_table(conn, race_make_table)
create_table(conn, race_meta_make_table)

In [ ]:
def csv_to_postgres(conn, csv_name, table):
    f = open(csv_name, 'r')
    cursor = conn.cursor()
    try:
        cursor.copy_from(f, table, sep=",")
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print(f'{table} is done')
    cursor.close()

In [ ]:
horse_df['name'] = horse_df['name'].apply(lambda x: str(x).replace(',', ''))
horse_df['stable_name'] = horse_df['stable_name'].apply(lambda x: str(x).replace(',', ''))

In [ ]:
# horse_df.to_csv('data/horse_df_upload.csv', index=False, header=False)
# race_df.drop(columns = ['odds']).dropna().to_csv('data/horse_race_data_db_upload.csv', index=False, header=False)
race_df3['gate'] = race_df3['gate'].fillna(0)
race_df3.drop(columns = ['odds']).dropna().to_csv('data/horse_race_data_db_upload3.csv', index=False, header=False)

In [131]:
# race_meta_df['name'] = race_meta_df['name'].apply(lambda x: str(x).replace(',', ''))
# race_meta_df['event_type'] = race_meta_df['event_type'].fillna('')
# race_meta_df.to_csv('data/race_meta_df_upload.csv', index=False, header=False)

race_meta_df2.insert(3, 'event_type', race_meta_df2.shape[0]*[''])
race_meta_df2['name'] = race_meta_df2['name'].apply(lambda x: str(x).replace(',', ''))
race_meta_df2['event_type'] = race_meta_df2['event_type'].fillna('')
race_meta_df2.to_csv('data/race_meta_df_upload2.csv', index=False, header=False)

In [ ]:
csv_to_postgres(conn, 'data/horse_df_upload.csv', 'horses')

In [ ]:
csv_to_postgres(conn, 'data/horse_race_data_db_upload.csv', 'races')

In [ ]:
csv_to_postgres(conn, 'data/race_meta_df_upload.csv', 'races_info')

In [132]:
csv_to_postgres(conn, 'data/race_meta_df_upload2.csv', 'races_info')

races_info is done


In [ ]:
csv_to_postgres(conn, 'data/race_meta_df_upload2.csv', 'races_info')

In [ ]:
csv_to_postgres(conn, 'data/horse_race_data_db_upload2.csv', 'races')

In [ ]:
csv_to_postgres(conn, 'data/horse_race_data_db_upload3.csv', 'races')

In [ ]:
conn.close()

In [210]:
from flask import jsonify, Flask

app = Flask(__name__)

In [218]:
with app.app_context():
    print(jsonify([{'a':1},{'b':1}]))

<Response 18 bytes [200 OK]>
